In [1]:

import json
import sys
import re
import time
import warnings
import requests
import re
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

warnings.filterwarnings('ignore')

In [2]:
# Function to get video URLs from the "Videos" tab of a YouTube channel
def get_video_urls(channel_url):
    # Append "/videos" to the channel URL
    channel_url += '/videos'

    # Send a GET request to the channel's "Videos" tab URL
    response = requests.get(channel_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Use regular expression to find video URLs containing "videoId"
        video_urls = re.findall(r'"videoId":"([^"]+)"', response.text)

        # Construct complete video URLs
        #video_urls = ['https://www.youtube.com/watch?v=VKBnCrPoF64' + video_id for video_id in video_urls]
        video_urls = ['https://www.youtube.com/watch?v=' + video_id for video_id in video_urls]

        # Remove duplicates
        video_urls = list(set(video_urls))

        return video_urls
    else:
        print('Failed to retrieve channel page. Status code:', response.status_code)
        return None

# Example usage
channel_urls = ["https://www.youtube.com/@callofdutymobile","https://www.youtube.com/@CODMesports", "https://www.youtube.com/@DeepanshuCODYT"]
video_urls = []
#channel_url = 'https://www.youtube.com/@IGN'  ### ENTER THE YOUTUBE CHANNEL'S LINK HERE ###
for channel_url in channel_urls:
  temp_urls = get_video_urls(channel_url)
  for temp in temp_urls:
    video_urls.append(temp)

if video_urls:
    #print('Video URLs in the "Videos" tab of the channel (without duplicates):')
    for url in video_urls:
        print(url)

https://www.youtube.com/watch?v=M73hdSxW1sM
https://www.youtube.com/watch?v=siM4W-4nuMc
https://www.youtube.com/watch?v=ixGaMxHs6A4
https://www.youtube.com/watch?v=kni1w-JxViw
https://www.youtube.com/watch?v=VY6mcZmetYU
https://www.youtube.com/watch?v=LE4SpImAQdY
https://www.youtube.com/watch?v=EtOtuQmaljU
https://www.youtube.com/watch?v=6u2wdQNxs-Q
https://www.youtube.com/watch?v=2rL4foKjtsA
https://www.youtube.com/watch?v=zlmI7ioQ_iY
https://www.youtube.com/watch?v=NatBX3VjnWw
https://www.youtube.com/watch?v=WHLkF566Jn4
https://www.youtube.com/watch?v=MW0UGCIg1tU
https://www.youtube.com/watch?v=MaBiS5JvoYc
https://www.youtube.com/watch?v=CPDxA69zbi8
https://www.youtube.com/watch?v=GZi0Y4HvOS8
https://www.youtube.com/watch?v=NO4ikM9CfYY
https://www.youtube.com/watch?v=Fj6W2iMtKwE
https://www.youtube.com/watch?v=UYvKs8b74qs
https://www.youtube.com/watch?v=ms8wiprrdns
https://www.youtube.com/watch?v=M_VX_8JIe_A
https://www.youtube.com/watch?v=NDYG3j24zko
https://www.youtube.com/watch?v=

In [3]:
print(len(video_urls))

90


In [50]:
YOUTUBE_COMMENTS_AJAX_URL = 'https://www.youtube.com/comment_service_ajax'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'

# filter comments by popularity or recent, 0:False, 1:True
SORT_BY_POPULAR = 0
# set comment limit
COMMENT_LIMIT = 100

YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

def regex_search(text, pattern, group=1, default=None):
    match = re.search(pattern, text)
    return match.group(group) if match else default


def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com' + endpoint['commandMetadata']['webCommandMetadata']['apiUrl']

    data = {'context': ytcfg['INNERTUBE_CONTEXT'],
            'continuation': endpoint['continuationCommand']['token']}

    for _ in range(retries):
        response = session.post(url, params={'key': ytcfg['INNERTUBE_API_KEY']}, json=data)
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403, 413]:
            return {}
        else:
            time.sleep(sleep)


def download_comments(YOUTUBE_VIDEO_URL, language=None, sleep=0.1):
    session = requests.Session()
    session.headers['User-Agent'] = USER_AGENT
    response = session.get(YOUTUBE_VIDEO_URL)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT', 'YES+cb', domain='.youtube.com')
        response = session.get(YOUTUBE_VIDEO_URL)

    html = response.text
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:
        return  # Unable to extract configuration
    if language:
        ytcfg['INNERTUBE_CONTEXT']['client']['hl'] = language

    data = json.loads(regex_search(html, YT_INITIAL_DATA_RE, default=''))

    section = next(search_dict(data, 'itemSectionRenderer'), None)
    renderer = next(search_dict(section, 'continuationItemRenderer'), None) if section else None
    if not renderer:
        # Comments disabled?
        return

    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session, continuation, ytcfg)

        if not response:
            break
        if list(search_dict(response, 'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + next(search_dict(response, 'externalErrorMessage')))

        actions = list(search_dict(response, 'reloadContinuationItemsCommand')) + \
                  list(search_dict(response, 'appendContinuationItemsAction'))
        for action in actions:
            for item in action.get('continuationItems', []):
                if action['targetId'] == 'comments-section':
                    # Process continuations for comments and replies.
                    continuations[:0] = [ep for ep in search_dict(item, 'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # Process the 'Show more replies' button
                    continuations.append(next(search_dict(item, 'buttonRenderer'))['command'])

        for comment in reversed(list(search_dict(response, 'commentRenderer'))):
            # Print comment data for debugging
            #print("Comment Data:", comment)
            yield {
                   'text': ''.join([c['text'] for c in comment['contentText'].get('runs', [])]),
                   'author': comment.get('authorText', {}).get('simpleText'),
                   'votes': comment.get('voteCount', {}).get('simpleText', '0'),
                   'dislikes': comment.get('dislikeCount', {}).get('simpleText', '0'),
                   'replies': comment.get('replyCount'),
                   'id': comment.get('commentId'),
                   # Add more fields as needed
                  }

        time.sleep(sleep)



def search_dict(partial, search_key):
    stack = [partial]
    while stack:
        current_item = stack.pop()
        if isinstance(current_item, dict):
            for key, value in current_item.items():
                if key == search_key:
                    yield value
                else:
                    stack.append(value)
        elif isinstance(current_item, list):
            for value in current_item:
                stack.append(value)

#Change code here
def main(url, schema, spark):
    #spark = SparkSession.builder.appName("YouTubeComments").getOrCreate()
    #schema = StructType().add("comment_id", StringType()).add("comment_text", StringType()).add("author", StringType()).add("likes", StringType()).add("timestamp", StringType())

    try:
        youtube_url = url
        limit = COMMENT_LIMIT
        count = 0
        start_time = time.time()

        df_comment = spark.createDataFrame([], schema=schema)

        for comment in download_comments(youtube_url):
            #mod_comment = spark.createDataFrame([comment], schema=schema)
            #df_comment = df_comment.union(mod_comment)

            id = comment['id']
            comment_text = comment['text']
            author = comment['author']
            likes = int(comment['votes'])
            dislikes = int(comment['dislikes'])
            replies = comment['replies']
            df_comment = df_comment.union(spark.createDataFrame([(comment_text, author, likes, dislikes, replies, id)], schema=schema))



            count += 1
            if limit and count >= limit:
                break
        return df_comment

    except Exception as e:
        print('Error:', str(e))
        sys.exit(1)

In [51]:
# Starting a spark session
spark = SparkSession.builder.appName("YouTubeComments").getOrCreate()
schema = StructType().add("comment_text", StringType()).add("comment_author", StringType()).add("votes", IntegerType(), nullable=True).add("dislikes", IntegerType(), nullable=True).add("replies", IntegerType(), nullable=True).add("id", StringType())

#dataset = pd.DataFrame()
dataset = spark.createDataFrame([], schema=schema)

for video in video_urls:
  extradf = main(video, schema, spark)
  dataset = dataset.union(extradf)
#dataset.reset_index(drop=True, inplace=True)


In [59]:
dataset.show(20)

24/04/04 14:40:06 WARN DAGScheduler: Broadcasting large task binary with size 7.0 MiB
24/04/04 14:40:06 WARN DAGScheduler: Broadcasting large task binary with size 7.0 MiB
24/04/04 14:40:06 WARN DAGScheduler: Broadcasting large task binary with size 7.0 MiB
24/04/04 14:40:08 WARN DAGScheduler: Broadcasting large task binary with size 7.0 MiB
24/04/04 14:40:12 WARN DAGScheduler: Broadcasting large task binary with size 7.0 MiB


+--------------------+--------------------+-----+--------+-------+--------------------+
|        comment_text|      comment_author|votes|dislikes|replies|                  id|
+--------------------+--------------------+-----+--------+-------+--------------------+
|I like how codm p...|         @--FlameZ--|   51|       0|      3|UgyTUAWhSyj0QI6Pz...|
|Please add the we...|       @Angel--AR-15|    7|       0|   NULL|Ugy_rJN270SWTrHvt...|
|Blitz Is Amazing ...|        @gaminguceyt|    2|       0|   NULL|Ugx82pUfNqR-GWaig...|
|I'm just saying i...|      @jaevaldez9650|    1|       0|   NULL|UgxtOCQDBmdL2bZ7p...|
|Button to block m...|       @Martin-zb4rk|    3|       0|      3|UgzJsAoP-3kzZmLh9...|
|Valentine's Day i...|  @morganmeliora2603|    1|       0|   NULL|UgxABlusdyDN8i5Lh...|
|Hello to the Acti...|            @M-ev1zz|    0|       0|   NULL|Ugzlwo2nHEgOUc2yB...|
|Would be nice to ...|       @KingPepper41|    4|       0|   NULL|UgxSjyCV-jwbpv7gX...|
|Still waiting for...|   @hunter

In [53]:
dataset.count()

24/04/04 14:32:06 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


800

In [55]:
from pyspark.sql.functions import regexp_replace
dataset = dataset.withColumn("comment_text", regexp_replace("comment_text", "[\n\r]", ""))

In [56]:
youtubePath = "hdfs://10.123.51.194/user/g23/spark_hdfs_webScrapingv3.csv"
#dataset.write.csv(youtubePath, header=True)
dataset.write.option("header",True).csv(youtubePath)

24/04/04 14:36:28 WARN DAGScheduler: Broadcasting large task binary with size 6.5 MiB


In [62]:
yt_df = spark.read.csv("hdfs://10.123.51.194/user/g23/spark_hdfs_webScrapingv3.csv", header=True)
yt_df.show(20)
#yt_pdf = yt_df.toPandas()
#local_file_path = "/home/g23/ASSIGNMENT CODES/test2.csv"
#yt_pdf.to_csv(local_file_path, index=False)

+--------------------+--------------------+-----+--------+-------+--------------------+
|        comment_text|      comment_author|votes|dislikes|replies|                  id|
+--------------------+--------------------+-----+--------+-------+--------------------+
|Just wanted to sa...|          @J3FFBezos|   11|       0|      1|UgwNKGEArXPJaKx-y...|
|The first LAN eve...|       @josepila9310|    5|       0|   NULL|UgxY1qWMgGKDLtXZv...|
|EXCELENTE CAMPEÓN...|@gerardomartinez6063|    0|       0|   NULL|UgzR-GPIW5LZERY4a...|
|Yrr me ye nhi sam...|            @Amaze__|    1|       0|   NULL|UgwjTC9JHr9ux3lYv...|
|👑👑👑👑👑👑👑👑?...|@gerardomartinez6063|    0|       0|   NULL|UgwScr2cyFWXV-Ff3...|
|Just curious wasn...|     @samuelconn3571|    0|       0|   NULL|Ugx7soqAm0113R5ju...|
|he always know wh...|       @cjkamela4896|    2|       0|   NULL|Ugx1leLq0i0wtMPlh...|
|Why do not you do...|   @silverbullet4966|    0|       0|   NULL|UgyuuSJVHHUP6Go4N...|
|Bring back the ak...|            @gads9

In [63]:
yt_df.count()

800

In [64]:
spark.stop()